In [1]:
# Load modules for data manipulation
from dotenv import load_dotenv
import seaborn as sn
import matplotlib.pyplot as plt
from subprocess import call
import pandas as pd
import numpy as np
import os

In [2]:
# Load modules for machine learning
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Calculate the accurancy of the model
from sklearn.metrics import accuracy_score, classification_report

# For drawing the graph
from sklearn.tree import export_graphviz

In [3]:
load_dotenv()

True

In [4]:
eco = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'eco-products.csv', encoding='ISO-8859-1', sep='\t')
# clt = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ38.csv', encoding='ISO-8859-1', sep='\t')
clt = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TR35.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)
ctr = pd.read_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ7S.csv', encoding='ISO-8859-1', sep='\t', low_memory=False)

## Get the best ecological product

In [112]:
eco = [str(i).zfill(5) for i in eco['COPRO']]

In [9]:
## ECO: 00003 | 242568 - LIVRET DEVELOPPEMENT DURABLE ET SOLIDAIRE
## DEPOTS MONETAIRES - EPARGNE MONETAIRE LIQUIDE - LIVRETS REGLEMENTES
ctr[ctr['COPRO'].isin(eco)]['COPRO'].value_counts()

00003    242568
00546      5141
00548      4785
07649      2724
00940      2166
00509      1876
00547      1027
00941       388
03992       101
00565        83
06458        75
Name: COPRO, dtype: int64

## Concatinate with other tables TJ39 - TJDR - TJER

In [5]:
tj39 = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj39.csv', encoding='ISO-8859-1', sep=';')
tjdr = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjdr.csv', encoding='ISO-8859-1', sep=';')
tjer = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tjer.csv', encoding='ISO-8859-1', sep=';')

In [6]:
tj39 = tj39[['COMAX', 'MSMENC', 'MTPATR', 'MTVAOP']]
tjdr = tjdr[['COMAX', 'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI']]
tjer = tjer[['COMAX', 'MTAPJE', 'MTCPJE', 'MTPJE']]

In [7]:
tj39['COMAX'] = [val[:10] for val in tj39['COMAX'].values]

In [8]:
# pour le cas de la table tj39, la variable COMAX elle est de 13 caracteres
# or celle des autres tables sont de 10, donc il fallait convertir de 13 a 10
# on prend les 10 premiers caractères du COMAX de tj39, d'apres le data engineer
# de la PBS.
clt[clt['COMAX'] == '1ce15b6b35']

,COMAX,CTCOPO,COESPF,COGRRB,COSGPA,CTMENB,COCINS,COPOST,CTSIFA,CTSCPI,...,QCPREV,QCIARD,MTFLPR,MTFLCO,MTRSFI,MTRSMO,QCFPRORB,QFPROSGR,LAT,LON
22460,1ce15b6b35,013,b5ba55db2a,96d8b7597a,3200,07,66028,66330.0,5,2300.0,...,NaN,NaN,NaN,NaN,16365.49,13395.05,NaN,NaN,42.6813,2.9412


In [9]:
# This table is only good fot the moral clients only
data = pd.merge(clt,tj39,on='COMAX',how='left')
data = pd.merge(data,tjer,on='COMAX',how='left')
data = pd.merge(data,tjdr,on='COMAX',how='left')

In [12]:
data = data.dropna()

In [10]:
del data['MTCPJE']

In [11]:
data.shape

(1689052, 120)

In [23]:
data['CTSCPI'] = data['CTSCPI'].astype(str)
data['COPOST'] = data['COPOST'].astype(str)

data['COPOST'].replace('nan', np.nan, inplace=True)
data['CTSCPI'].replace('nan', np.nan, inplace=True)

data.dropna(subset=['COPOST'], inplace=True)
data.dropna(subset=['CTSCPI'], inplace=True)

In [25]:
data = data.drop_duplicates(subset="COMAX")

In [30]:
lll = ['MTCDIM', 'MTRECD', 'MTRETT', 'QCPRDI', 'QCPRCP', 'QCPRQU',
'QCPRFI', 'QCPRTR', 'QCPRAS', 'QCPRPR', 'MTELDD', 'MTECEL',
'MTEPEL', 'MTECSL', 'MTEPEP', 'MTELEP', 'MTELIJ', 'MTECIM',
'MTECCS', 'MTECEQ', 'MTECTR', 'MTEEML', 'MTEASV', 'MTECTO',
'MTECPA', 'MTE19D', 'MTE21D', 'MTEEMC', 'MTESOC', 'MCTOTA',
'MCTOTE', 'QTCLIF', 'QCBPFA', 'QCIARD', 'MTFLPR', 'MTFLCO',
'MTRSFI', 'MTRSMO', 'QCFPRORB', 'QFPROSGR', 'MSMENC', 'MTPATR', 'MTVAOP',
'MTRVIM', 'MTRVFR', 'QTPAFI', 'COHAVI', 'MTAPJE', 'MTPJE']

In [31]:
data[lll] = data[lll].fillna(data[lll].median())

/home/herharf/.local/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [65]:
data['COSEXE'].replace('M', 0, inplace=True)
data['COSEXE'].replace('F', 1, inplace=True)

/home/herharf/.local/lib/python3.6/site-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [72]:
data['COSEXE']

dtype('int64')

## Medel

In [119]:
cols = list(data.columns.values[1:-1])
cols.remove('COPOST')
cols.remove('COESPF')
cols.remove('COGRRB')

In [123]:
len(cols)

96

In [95]:
for l in cols[:-1]:
    try:
        data[l] = data[l].fillna(data[l].median())
    except:
        print(l)

/home/herharf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [97]:
data.isna().sum()

COMAX     0
COESPF    0
COGRRB    0
COPOST    0
CTSCPI    0
         ..
MTRVIM    0
MTRVFR    0
QTPAFI    0
COHAVI    0
ECO       0
Length: 101, dtype: int64

In [98]:
def done(ecos, cols):
    print('THIS IS PRODUCT:', ecos)
    
    # get the ids of the products
    ids = ctr[ctr['COPRO'].isin(ecos)]['COMAX'].values
    data['ECO'] = data['COMAX'].isin(ids)
    
    # change the types
    data['CTSCPI'] = data['CTSCPI'].astype(str)
    data['COPOST'] = data['COPOST'].astype(str)
    
    ''' The Model'''
    
    # Labels are the values we want to predict
    labels = np.array(data['ECO'])

    # Remove the labels from the features
    features = np.array(data[cols])

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

    # Train the model on training data
    rf.fit(train_features, train_labels)

    # Use the forest's predict method on the test data
    predictions = rf.predict(test_features)

    # Classification report
    print(classification_report(test_labels, predictions.round()))    

In [114]:
# get the ids of the products
ids = ctr[ctr['COPRO'].isin(eco)]['COMAX'].values
data['ECO'] = data['COMAX'].isin(ids)

/home/herharf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [121]:
''' The Model'''

# Labels are the values we want to predict
labels = np.array(data['ECO'])

# Remove the labels from the features
features = np.array(data[cols])

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [122]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Classification report
print(classification_report(test_labels, predictions.round()))

              precision    recall  f1-score   support

       False       0.90      0.99      0.95     69151
        True       0.98      0.77      0.87     32128

    accuracy                           0.92    101279
   macro avg       0.94      0.88      0.91    101279
weighted avg       0.93      0.92      0.92    101279



# - - - - - - - -- - - - - - - - - - - - - - - - -- - - - - - 

In [50]:
# Extract single tree
estimator = rf.estimators_[5]

export_graphviz(estimator, out_file='tree.dot', feature_names = cols,
                rounded = True, proportion = False, precision = 2, filled = True)

# '-Gdpi=600'
call(['dot', '-Tsvg', 'tree.dot', '-o', 'tree.svg'])

0

In [81]:
matrix = data.corr().values

In [82]:
for i in range(len(cols) - 1):
    for j in range(i+1, len(cols)):
        print('{} -- {}:: {}'.format(cols[i], cols[j], matrix[i][j]*100))

AGE -- COSEXE:: 3.072393652040798
AGE -- CTSCPI:: 7.987595669398723
AGE -- LAT:: -9.897809733166868
AGE -- LON:: -0.6471901958887913
AGE -- MSMENC:: -1.7420519784402444
AGE -- MTPATR:: 8.394401602945948
AGE -- MTVAOP:: 6.912120934920904
AGE -- MTAPJE:: 4.410312942447301
AGE -- MTPJE:: 1.1217326494827329
AGE -- MTRVIM:: -1.4391059265382167
AGE -- MTRVFR:: -6.744124485757126
AGE -- QTPAFI:: -7.625348572621979
AGE -- COHAVI:: 2.72763455403074
COSEXE -- CTSCPI:: 0.5374285254971073
COSEXE -- LAT:: -4.733964265404057
COSEXE -- LON:: -1.316769807298293
COSEXE -- MSMENC:: 0.3877980554603042
COSEXE -- MTPATR:: -0.40537251845024663
COSEXE -- MTVAOP:: -0.9525686433095442
COSEXE -- MTAPJE:: 2.52222228694067
COSEXE -- MTPJE:: 0.5455664141792879
COSEXE -- MTRVIM:: -2.4230034532981426
COSEXE -- MTRVFR:: -2.237319490326995
COSEXE -- QTPAFI:: -0.9777861999174383
COSEXE -- COHAVI:: 1.7008681154137766
CTSCPI -- LAT:: 0.6754989598428544
CTSCPI -- LON:: -1.7505804973627346
CTSCPI -- MSMENC:: 0.077062510675

In [69]:
cols[13]

'COHAVI'